<a href="https://colab.research.google.com/github/ZerXXX0/Cloud-classification-with-maxvit/blob/main/CCSN_GCD_cross_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

inaiwt_gcd_with_val_path = kagglehub.dataset_download('inaiwt/gcd-with-val')
inaiwt_gcd_ccsn_val_reduced_path = kagglehub.dataset_download('inaiwt/gcd-ccsn-val-reduced')

print('Data source import complete.')


In [ ]:
from urllib.request import urlopen
import timm
import torch
import zipfile,os
from PIL import Image
from pathlib import Path
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim

In [ ]:
train_dir_gcd = '/kaggle/input/gcd-ccsn-val-reduced/GCD val from Train/train'
test_dir_gcd =  '/kaggle/input/gcd-ccsn-val-reduced/GCD val from Train/test'
val_dir_gcd =  '/kaggle/input/gcd-ccsn-val-reduced/GCD val from Train/val'
train_dir_ccsn = '/kaggle/input/gcd-ccsn-val-reduced/CCSN Val from Train(80_20)/train'
test_dir_ccsn =  '/kaggle/input/gcd-ccsn-val-reduced/CCSN Val from Train(80_20)/test'
val_dir_ccsn =  '/kaggle/input/gcd-ccsn-val-reduced/CCSN Val from Train(80_20)/val'

Train in GCD

In [ ]:
# Load Pretrained Model
model = timm.create_model(
    'maxvit_tiny_tf_224.in1k',
    pretrained=True,
    num_classes=5,
)
device = torch.device("cuda")

# Modify the classifier to include dropout
class CustomModel(nn.Module):
    def __init__(self, base_model, dropout_rate=0.3):
        super(CustomModel, self).__init__()
        self.base_model = base_model
        in_features = base_model.head.fc.in_features  # Get original classifier input features
        self.base_model.head.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),  # Add dropout before final layer
            nn.Linear(in_features, 5)   # Output layer (5 classes)
        )

    def forward(self, x):
        return self.base_model(x)

# Wrap the model with dropout
dropout_model = CustomModel(model, dropout_rate=0.3)

# Enable multi-GPU support if available
if torch.cuda.device_count() > 1:
    dropout_model = nn.DataParallel(dropout_model)

dropout_model = dropout_model.eval()
dropout_model.to(device)

model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
trans = timm.data.create_transform(**data_config, is_training=False)

class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_names = os.listdir(data_dir)

        for label, class_name in enumerate(self.class_names):
            class_dir = os.path.join(data_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.images.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label  # Ensure this returns a tuple of (image, label)

#transform = transforms.Compose([
#    transforms.Resize((224, 224)),  # Resize images to 224x224
#    transforms.ToTensor(),           # Convert images to PyTorch tensors
#    transforms.RandomHorizontalFlip(),
#    transforms.RandomRotation(10),
#    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
#])

transform = transforms.Compose([
    transforms.Resize((224, 224)),                     # Resize to match model input size
    #transforms.RandomHorizontalFlip(p=0.5),           # Flip horizontally (clouds are often symmetrical)
    #transforms.RandomVerticalFlip(p=0.3),             # Flip vertically (useful for aerial/satellite views)
    #transforms.RandomRotation(30),                    # Random rotations to simulate viewpoint changes
    #transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Randomly crop to add variability
    #transforms.RandomAutocontrast(),                  # Auto-contrast to normalize lighting
    #transforms.RandomAdjustSharpness(sharpness_factor=2),  # Enhance sharpness for fine details
    transforms.ToTensor(),                            # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize for ImageNet-pretrained models
                         std=[0.229, 0.224, 0.225])
])

# Create an instance of the CustomDataset
dataset = CustomDataset(data_dir=train_dir_gcd, transform=transform)

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
for data in train_loader:
    print(data[0].shape)  # This will show you the structure of the data being returned
    inputs, targets = data  # Unpack only if it has the correct structure
    break


import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time
import csv

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")
model.to(device)
num_epochs = 10

#New with csv added
csv_file = "train_log_tiny_GCD.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Epoch", "Train Loss", "Train Accuracy","Val Loss", "Val Accuracy", "Time (seconds)", "Current LR"])

val_dataset = ImageFolder(root=val_dir_gcd, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# Scheduler
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Training loop
model.train()  # Set the model to training mode
for epoch in range(num_epochs):
    start_time = time.time()
    correct = 0
    total = 0
    running_loss = 0.0

    progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")

    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)  # Move to device

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

         # Calculate metrics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    # Calculate epoch metrics
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total * 100

    # Validation
    model.eval();
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for val_inputs, val_targets in val_loader:
            val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_targets).item()
            _, val_predicted = torch.max(val_outputs, 1)
            val_total += val_targets.size(0)
            val_correct += (val_predicted == val_targets).sum().item()

    val_loss /= len(val_loader)
    val_acc = val_correct / val_total * 100

    #scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']

    epoch_time = time.time() - start_time

    # Log results to CSV
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([epoch + 1, epoch_loss, epoch_acc, val_loss, val_acc, epoch_time, current_lr])

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, Time: {epoch_time:.2f}s")
    print(f"Current Learning Rate: {current_lr:.6f}")
    model.train()

torch.Size([32, 3, 224, 224])


Epoch [1/10]: 100%|██████████| 23/23 [00:20<00:00,  1.15it/s]


Epoch [1/10], Train Loss: 0.9412, Train Acc: 68.76%, Val Loss: 2.4692, Val Acc: 2.46%, Time: 37.70s
Current Learning Rate: 0.000100


Epoch [2/10]: 100%|██████████| 23/23 [00:19<00:00,  1.16it/s]


Epoch [2/10], Train Loss: 0.3504, Train Acc: 88.28%, Val Loss: 3.0597, Val Acc: 7.16%, Time: 36.96s
Current Learning Rate: 0.000100


...

Epoch [8/10], Train Loss: 0.0366, Train Acc: 99.02%, Val Loss: 4.7671, Val Acc: 4.37%, Time: 36.85s
Current Learning Rate: 0.000100


Epoch [9/10]: 100%|██████████| 23/23 [00:19<00:00,  1.18it/s]


Epoch [9/10], Train Loss: 0.1103, Train Acc: 96.09%, Val Loss: 4.5737, Val Acc: 7.98%, Time: 36.66s
Current Learning Rate: 0.000100


Epoch [10/10]: 100%|██████████| 23/23 [00:19<00:00,  1.17it/s]


Epoch [10/10], Train Loss: 0.0880, Train Acc: 97.21%, Val Loss: 4.5922, Val Acc: 5.53%, Time: 36.69s
Current Learning Rate: 0.000100


Test in CCSN

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),                   # Resize to match the model input
    transforms.ToTensor(),                           # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])  # Normalize (same as training)
])

test_dataset = ImageFolder(root=test_dir_ccsn, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# 4. Define testing loop
correct_predictions = 0
total_samples = 0
all_targets = []
all_preds = []

model.eval()

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, dim=1)

        correct_predictions += (preds == targets).sum().item()
        total_samples += targets.size(0)

        all_targets.extend(targets.tolist())
        all_preds.extend(preds.tolist())

accuracy = accuracy_score(all_targets, all_preds)
precision = precision_score(all_targets, all_preds, average=None, zero_division=0)
recall = recall_score(all_targets, all_preds, average=None, zero_division=0)
conf_matrix = confusion_matrix(all_targets, all_preds)
report = classification_report(all_targets, all_preds, zero_division=0)

results_file = f"Tiny train in GCD test in ccsn.txt"
with open(results_file, "w") as f:
    f.write(f"Accuracy: {accuracy * 100:.2f}%\n")
    f.write(f"\nPrecision (per class): {precision}\n")
    f.write(f"\nRecall (per class): {recall}\n")
    f.write(f"\nConfusion Matrix:\n{conf_matrix}\n")
    f.write(f"\nClassification Report:\n{report}\n")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(conf_matrix)
print(classification_report)

Accuracy: 22.77%
Precision: [0.18181818 0.2        0.38461538 0.14285714 0.21917808]
Recall: [0.44444444 0.04545455 0.53571429 0.02941176 0.33333333]
[[16  0  5  0 15]
 [13  2  1 10 18]
 [ 6  1 15  1  5]
 [30  7 10  2 19]
 [23  0  8  1 16]]
<function classification_report at 0x7e895c7ec940>


Test in GCD

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),                   # Resize to match the model input
    transforms.ToTensor(),                           # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])  # Normalize (same as training)
])

test_dataset = ImageFolder(root=test_dir_gcd, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# 4. Define testing loop
correct_predictions = 0
total_samples = 0
all_targets = []
all_preds = []

model.eval()

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, dim=1)

        correct_predictions += (preds == targets).sum().item()
        total_samples += targets.size(0)

        all_targets.extend(targets.tolist())
        all_preds.extend(preds.tolist())

accuracy = accuracy_score(all_targets, all_preds)
precision = precision_score(all_targets, all_preds, average=None, zero_division=0)
recall = recall_score(all_targets, all_preds, average=None, zero_division=0)
conf_matrix = confusion_matrix(all_targets, all_preds)
report = classification_report(all_targets, all_preds, zero_division=0)

results_file = f"Tiny train in GCD test in GCD.txt"
with open(results_file, "w") as f:
    f.write(f"Accuracy: {accuracy * 100:.2f}%\n")
    f.write(f"\nPrecision (per class): {precision}\n")
    f.write(f"\nRecall (per class): {recall}\n")
    f.write(f"\nConfusion Matrix:\n{conf_matrix}\n")
    f.write(f"\nClassification Report:\n{report}\n")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(conf_matrix)
print(classification_report)

Train in CCSN

In [ ]:
# Load Pretrained Model
model = timm.create_model(
    'maxvit_tiny_tf_224.in1k',
    pretrained=True,
    num_classes=5,
)
device = torch.device("cuda")

# Modify the classifier to include dropout
class CustomModel(nn.Module):
    def __init__(self, base_model, dropout_rate=0.3):
        super(CustomModel, self).__init__()
        self.base_model = base_model
        in_features = base_model.head.fc.in_features  # Get original classifier input features
        self.base_model.head.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),  # Add dropout before final layer
            nn.Linear(in_features, 5)   # Output layer (5 classes)
        )

    def forward(self, x):
        return self.base_model(x)

# Wrap the model with dropout
dropout_model = CustomModel(model, dropout_rate=0.3)

# Enable multi-GPU support if available
if torch.cuda.device_count() > 1:
    dropout_model = nn.DataParallel(dropout_model)

dropout_model = dropout_model.eval()
dropout_model.to(device)

model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
trans = timm.data.create_transform(**data_config, is_training=False)

class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_names = os.listdir(data_dir)

        for label, class_name in enumerate(self.class_names):
            class_dir = os.path.join(data_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.images.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label  # Ensure this returns a tuple of (image, label)

#transform = transforms.Compose([
#    transforms.Resize((224, 224)),  # Resize images to 224x224
#    transforms.ToTensor(),           # Convert images to PyTorch tensors
#    transforms.RandomHorizontalFlip(),
#    transforms.RandomRotation(10),
#    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
#])

transform = transforms.Compose([
    transforms.Resize((224, 224)),                     # Resize to match model input size
    #transforms.RandomHorizontalFlip(p=0.5),           # Flip horizontally (clouds are often symmetrical)
    #transforms.RandomVerticalFlip(p=0.3),             # Flip vertically (useful for aerial/satellite views)
    #transforms.RandomRotation(30),                    # Random rotations to simulate viewpoint changes
    #transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Randomly crop to add variability
    #transforms.RandomAutocontrast(),                  # Auto-contrast to normalize lighting
    #transforms.RandomAdjustSharpness(sharpness_factor=2),  # Enhance sharpness for fine details
    transforms.ToTensor(),                            # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize for ImageNet-pretrained models
                         std=[0.229, 0.224, 0.225])
])

# Create an instance of the CustomDataset
dataset = CustomDataset(data_dir=train_dir_ccsn, transform=transform)

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
for data in train_loader:
    print(data[0].shape)  # This will show you the structure of the data being returned
    inputs, targets = data  # Unpack only if it has the correct structure
    break


import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time
import csv

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")
model.to(device)
num_epochs = 10

#New with csv added
csv_file = "train_log_tiny_ccsn.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Epoch", "Train Loss", "Train Accuracy","Val Loss", "Val Accuracy", "Time (seconds)", "Current LR"])

val_dataset = ImageFolder(root=val_dir_gcd, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# Scheduler
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Training loop
model.train()  # Set the model to training mode
for epoch in range(num_epochs):
    start_time = time.time()
    correct = 0
    total = 0
    running_loss = 0.0

    progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")

    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)  # Move to device

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

         # Calculate metrics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    # Calculate epoch metrics
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total * 100

    # Validation
    model.eval();
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for val_inputs, val_targets in val_loader:
            val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_targets).item()
            _, val_predicted = torch.max(val_outputs, 1)
            val_total += val_targets.size(0)
            val_correct += (val_predicted == val_targets).sum().item()

    val_loss /= len(val_loader)
    val_acc = val_correct / val_total * 100

    #scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']

    epoch_time = time.time() - start_time

    # Log results to CSV
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([epoch + 1, epoch_loss, epoch_acc, val_loss, val_acc, epoch_time, current_lr])

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, Time: {epoch_time:.2f}s")
    print(f"Current Learning Rate: {current_lr:.6f}")
    model.train()

torch.Size([32, 3, 224, 224])


Epoch [1/10]: 100%|██████████| 23/23 [00:19<00:00,  1.15it/s]


Epoch [1/10], Train Loss: 1.3352, Train Acc: 47.42%, Val Loss: 1.8198, Val Acc: 13.37%, Time: 37.22s
Current Learning Rate: 0.000100


Epoch [2/10]: 100%|██████████| 23/23 [00:19<00:00,  1.18it/s]


Epoch [2/10], Train Loss: 0.7299, Train Acc: 76.57%, Val Loss: 1.9857, Val Acc: 20.87%, Time: 36.34s
Current Learning Rate: 0.000100


...

Epoch [8/10], Train Loss: 0.0557, Train Acc: 98.74%, Val Loss: 3.1203, Val Acc: 21.83%, Time: 35.73s
Current Learning Rate: 0.000100


Epoch [9/10]: 100%|██████████| 23/23 [00:19<00:00,  1.20it/s]


Epoch [9/10], Train Loss: 0.0519, Train Acc: 98.19%, Val Loss: 2.8165, Val Acc: 28.04%, Time: 35.87s
Current Learning Rate: 0.000100


Epoch [10/10]: 100%|██████████| 23/23 [00:19<00:00,  1.20it/s]


Epoch [10/10], Train Loss: 0.0330, Train Acc: 98.61%, Val Loss: 2.6589, Val Acc: 26.74%, Time: 36.04s
Current Learning Rate: 0.000100


Test in CCSN

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),                   # Resize to match the model input
    transforms.ToTensor(),                           # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])  # Normalize (same as training)
])

test_dataset = ImageFolder(root=test_dir_ccsn, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# 4. Define testing loop
correct_predictions = 0
total_samples = 0
all_targets = []
all_preds = []

model.eval()

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, dim=1)

        correct_predictions += (preds == targets).sum().item()
        total_samples += targets.size(0)

        all_targets.extend(targets.tolist())
        all_preds.extend(preds.tolist())

accuracy = accuracy_score(all_targets, all_preds)
precision = precision_score(all_targets, all_preds, average=None, zero_division=0)
recall = recall_score(all_targets, all_preds, average=None, zero_division=0)
conf_matrix = confusion_matrix(all_targets, all_preds)
report = classification_report(all_targets, all_preds, zero_division=0)

results_file = f"Tiny train in CCSN test in ccsn.txt"
with open(results_file, "w") as f:
    f.write(f"Accuracy: {accuracy * 100:.2f}%\n")
    f.write(f"\nPrecision (per class): {precision}\n")
    f.write(f"\nRecall (per class): {recall}\n")
    f.write(f"\nConfusion Matrix:\n{conf_matrix}\n")
    f.write(f"\nClassification Report:\n{report}\n")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(conf_matrix)
print(classification_report)

Test in GCD

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),                   # Resize to match the model input
    transforms.ToTensor(),                           # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])  # Normalize (same as training)
])

test_dataset = ImageFolder(root=test_dir_gcd, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# 4. Define testing loop
correct_predictions = 0
total_samples = 0
all_targets = []
all_preds = []

model.eval()

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, dim=1)

        correct_predictions += (preds == targets).sum().item()
        total_samples += targets.size(0)

        all_targets.extend(targets.tolist())
        all_preds.extend(preds.tolist())

accuracy = accuracy_score(all_targets, all_preds)
precision = precision_score(all_targets, all_preds, average=None, zero_division=0)
recall = recall_score(all_targets, all_preds, average=None, zero_division=0)
conf_matrix = confusion_matrix(all_targets, all_preds)
report = classification_report(all_targets, all_preds, zero_division=0)

results_file = f"Tiny train in CCSN test in gcd.txt"
with open(results_file, "w") as f:
    f.write(f"Accuracy: {accuracy * 100:.2f}%\n")
    f.write(f"\nPrecision (per class): {precision}\n")
    f.write(f"\nRecall (per class): {recall}\n")
    f.write(f"\nConfusion Matrix:\n{conf_matrix}\n")
    f.write(f"\nClassification Report:\n{report}\n")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(conf_matrix)
print(classification_report)

Accuracy: 10.12%
Precision: [0.22222222 0.48837209 0.14285714 0.07560628 0.05714286]
Recall: [0.05405405 0.03250774 0.20689655 0.76811594 0.04081633]
[[  2   3   0  10  22]
 [  2  21  34 578  11]
 [  0   2   6  21   0]
 [  1  13   2  53   0]
 [  4   4   0  39   2]]
<function classification_report at 0x7e895c7ec940>


Train in both

In [ ]:
# Load Pretrained Model
model = timm.create_model(
    'maxvit_small_tf_224.in1k',
    pretrained=True,
    num_classes=5,
)
device = torch.device("cuda")

# Modify the classifier to include dropout
class CustomModel(nn.Module):
    def __init__(self, base_model, dropout_rate=0.3):
        super(CustomModel, self).__init__()
        self.base_model = base_model
        in_features = base_model.head.fc.in_features  # Get original classifier input features
        self.base_model.head.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),  # Add dropout before final layer
            nn.Linear(in_features, 5)   # Output layer (5 classes)
        )

    def forward(self, x):
        return self.base_model(x)

# Wrap the model with dropout
dropout_model = CustomModel(model, dropout_rate=0.3)

# Enable multi-GPU support if available
if torch.cuda.device_count() > 1:
    dropout_model = nn.DataParallel(dropout_model)

dropout_model = dropout_model.eval()
dropout_model.to(device)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
trans = timm.data.create_transform(**data_config, is_training=False)

class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_names = os.listdir(data_dir)

        for label, class_name in enumerate(self.class_names):
            class_dir = os.path.join(data_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.images.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label  # Ensure this returns a tuple of (image, label)

#transform = transforms.Compose([
#    transforms.Resize((224, 224)),  # Resize images to 224x224
#    transforms.ToTensor(),           # Convert images to PyTorch tensors
#    transforms.RandomHorizontalFlip(),
#    transforms.RandomRotation(10),
#    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
#])

transform = transforms.Compose([
    transforms.Resize((224, 224)),                     # Resize to match model input size
    #transforms.RandomHorizontalFlip(p=0.5),           # Flip horizontally (clouds are often symmetrical)
    #transforms.RandomVerticalFlip(p=0.3),             # Flip vertically (useful for aerial/satellite views)
    #transforms.RandomRotation(30),                    # Random rotations to simulate viewpoint changes
    #transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Randomly crop to add variability
    #transforms.RandomAutocontrast(),                  # Auto-contrast to normalize lighting
    #transforms.RandomAdjustSharpness(sharpness_factor=2),  # Enhance sharpness for fine details
    transforms.ToTensor(),                            # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize for ImageNet-pretrained models
                         std=[0.229, 0.224, 0.225])
])

# Create datasets for both sources
dataset_1 = CustomDataset(data_dir=train_dir_gcd, transform=transform)
dataset_2 = CustomDataset(data_dir=train_dir_ccsn, transform=transform)

val_dataset_1 = ImageFolder(root=val_dir_gcd, transform=transform)
val_dataset_2 = ImageFolder(root=val_dir_ccsn, transform=transform)

dataset = torch.utils.data.ConcatDataset([dataset_1, dataset_2])
val_dataset = torch.utils.data.ConcatDataset([val_dataset_1, val_dataset_2])


# Create an instance of the CustomDataset
#dataset = CustomDataset(data_dir=train_dir_gcd, transform=transform)

train_loader = DataLoader(dataset, batch_size=16, shuffle=True)
for data in train_loader:
    print(data[0].shape)  # This will show you the structure of the data being returned
    inputs, targets = data  # Unpack only if it has the correct structure
    break


import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time
import csv

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")
model.to(device)
num_epochs = 10

#New with csv added
csv_file = "train_log_small_GCD_ccsn_both.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Epoch", "Train Loss", "Train Accuracy","Val Loss", "Val Accuracy", "Time (seconds)", "Current LR"])

val_dataset = ImageFolder(root=val_dir_gcd, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

# Scheduler
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=3, verbose=True)

# Training loop
model.train()  # Set the model to training mode
for epoch in range(num_epochs):
    start_time = time.time()
    correct = 0
    total = 0
    running_loss = 0.0

    progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")

    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)  # Move to device

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

         # Calculate metrics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    # Calculate epoch metrics
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total * 100

    # Validation
    model.eval();
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for val_inputs, val_targets in val_loader:
            val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_targets).item()
            _, val_predicted = torch.max(val_outputs, 1)
            val_total += val_targets.size(0)
            val_correct += (val_predicted == val_targets).sum().item()

    val_loss /= len(val_loader)
    val_acc = val_correct / val_total * 100

    #scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']

    epoch_time = time.time() - start_time

    # Log results to CSV
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([epoch + 1, epoch_loss, epoch_acc, val_loss, val_acc, epoch_time, current_lr])

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, Time: {epoch_time:.2f}s")
    print(f"Current Learning Rate: {current_lr:.6f}")
    model.train()

torch.save(model.state_dict(), "Train in both.pth")

torch.Size([16, 3, 224, 224])


Epoch [1/10]: 100%|██████████| 90/90 [01:04<00:00,  1.40it/s]


Epoch [1/10], Train Loss: 0.9129, Train Acc: 68.27%, Val Loss: 3.2063, Val Acc: 13.92%, Time: 88.18s
Current Learning Rate: 0.000100


Epoch [2/10]: 100%|██████████| 90/90 [01:03<00:00,  1.41it/s]


Epoch [2/10], Train Loss: 0.3901, Train Acc: 85.70%, Val Loss: 3.9438, Val Acc: 11.60%, Time: 87.71s
Current Learning Rate: 0.000100


...

Epoch [8/10], Train Loss: 0.0397, Train Acc: 98.81%, Val Loss: 6.1884, Val Acc: 12.01%, Time: 87.49s
Current Learning Rate: 0.000100


Epoch [9/10]: 100%|██████████| 90/90 [01:03<00:00,  1.42it/s]


Epoch [9/10], Train Loss: 0.0688, Train Acc: 97.07%, Val Loss: 5.3030, Val Acc: 15.14%, Time: 87.49s
Current Learning Rate: 0.000100


Epoch [10/10]: 100%|██████████| 90/90 [01:03<00:00,  1.41it/s]


Epoch [10/10], Train Loss: 0.0781, Train Acc: 97.21%, Val Loss: 4.8328, Val Acc: 13.92%, Time: 87.62s
Current Learning Rate: 0.000100


Test in GCD

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),                   # Resize to match the model input
    transforms.ToTensor(),                           # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])  # Normalize (same as training)
])

test_dataset = ImageFolder(root=test_dir_gcd, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

# 4. Define testing loop
correct_predictions = 0
total_samples = 0
all_targets = []
all_preds = []

model.eval()

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, dim=1)

        correct_predictions += (preds == targets).sum().item()
        total_samples += targets.size(0)

        all_targets.extend(targets.tolist())
        all_preds.extend(preds.tolist())

accuracy = accuracy_score(all_targets, all_preds)
precision = precision_score(all_targets, all_preds, average=None, zero_division=0)
recall = recall_score(all_targets, all_preds, average=None, zero_division=0)
conf_matrix = confusion_matrix(all_targets, all_preds)
report = classification_report(all_targets, all_preds, zero_division=0)

results_file = f"Small train in both test in gcd.txt"
with open(results_file, "w") as f:
    f.write(f"Accuracy: {accuracy * 100:.2f}%\n")
    f.write(f"\nPrecision (per class): {precision}\n")
    f.write(f"\nRecall (per class): {recall}\n")
    f.write(f"\nConfusion Matrix:\n{conf_matrix}\n")
    f.write(f"\nClassification Report:\n{report}\n")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(conf_matrix)
print(classification_report)

Accuracy: 27.95%
Precision: [0.0703125  0.77865613 0.15151515 0.         0.02439024]
Recall: [0.24324324 0.30495356 0.86206897 0.         0.02040816]
[[  9   2   5   1  20]
 [ 72 197 115 242  20]
 [  1   3  25   0   0]
 [ 14  38  17   0   0]
 [ 32  13   3   0   1]]
<function classification_report at 0x7c1b5f0f4940>


Test in CCSN

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),                   # Resize to match the model input
    transforms.ToTensor(),                           # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])  # Normalize (same as training)
])

test_dataset = ImageFolder(root=test_dir_ccsn, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

# 4. Define testing loop
correct_predictions = 0
total_samples = 0
all_targets = []
all_preds = []

model.eval()

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, dim=1)

        correct_predictions += (preds == targets).sum().item()
        total_samples += targets.size(0)

        all_targets.extend(targets.tolist())
        all_preds.extend(preds.tolist())

accuracy = accuracy_score(all_targets, all_preds)
precision = precision_score(all_targets, all_preds, average=None, zero_division=0)
recall = recall_score(all_targets, all_preds, average=None, zero_division=0)
conf_matrix = confusion_matrix(all_targets, all_preds)
report = classification_report(all_targets, all_preds, zero_division=0)

results_file = f"Small train in both test in ccsn.txt"
with open(results_file, "w") as f:
    f.write(f"Accuracy: {accuracy * 100:.2f}%\n")
    f.write(f"\nPrecision (per class): {precision}\n")
    f.write(f"\nRecall (per class): {recall}\n")
    f.write(f"\nConfusion Matrix:\n{conf_matrix}\n")
    f.write(f"\nClassification Report:\n{report}\n")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(conf_matrix)
print(classification_report)

Accuracy: 22.77%
Precision: [0.16666667 0.11864407 0.54166667 0.20512821 0.25757576]
Recall: [0.16666667 0.15909091 0.46428571 0.11764706 0.35416667]
[[ 6  2  2  2 24]
 [ 1  7  2 14 20]
 [ 0  1 13 14  0]
 [ 3 47  5  8  5]
 [26  2  2  1 17]]
<function classification_report at 0x7c1b5f0f4940>
